In [721]:
import pandas as pd
from functools import reduce
import numpy as np
from collections import Counter
from unidecode import unidecode

In [722]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Load
 performance statistics from fbref.com are available in 5 separate csv files. Below we load them as dfs and name them.

In [723]:
overall = pd.read_csv("standard_stats.csv", header = [0,1])
chance_creation = pd.read_csv("goal_and_shot_csv", header = [0,1])
defense = pd.read_csv("defensive_actions.csv", header = [0,1])
passing = pd.read_csv("passing.csv", header = [0,1])
possession = pd.read_csv("possession.csv", header = [0,1])

In [724]:
overall.head()

Unnamed: 0_level_0                       Unnamed: 1_level_0  \
                  Rk                                   Player   
0                  1  Patrick van Aanholt\Patrick-van-Aanholt   
1                  2                    Max Aarons\Max-Aarons   
2                  3        Yunis Abdelhamid\Yunis-Abdelhamid   
3                  4    Suleiman Abdullahi\Suleiman-Abdullahi   
4                  5                  Mehdi Abeid\Mehdi-Abeid   

  Unnamed: 2_level_0 Unnamed: 3_level_0 Unnamed: 4_level_0  \
              Nation                Pos              Squad   
0             nl NED                 DF     Crystal Palace   
1            eng ENG                 DF       Norwich City   
2             ma MAR                 DF              Reims   
3             ng NGA               FWMF       Union Berlin   
4             dz ALG                 MF             Nantes   

   Unnamed: 5_level_0 Unnamed: 6_level_0 Unnamed: 7_level_0 Playing Time  \
                 Comp                Age               Born           MP   
0  eng Premier League               28.0             1990.0           29   
1  eng Premier League               19.0             2000.0           36   
2          fr Ligue 1               31.0             1987.0           28   
3       de Bundesliga               22.0             1996.0            6   
4          fr Ligue 1               26.0             1992.0           25   

               Performance                        Per 90 Minutes              \
  Starts   Min         Gls Ast PK PKatt CrdY CrdR            Gls   Ast   G+A   
0     29  2507           3   2  1     1    0    0           0.11  0.07  0.18   
1     36  3240           0   1  0     0    7    0           0.00  0.03  0.03   
2     28  2520           3   0  0     0    0    0           0.11  0.00  0.11   
3      1   157           1   0  0     0    1    0           0.57  0.00  0.57   
4     20  1817           1   2  0     0    5    0           0.05  0.10  0.15   

               Expected           Per 90 Minutes                            \
   G-PK G+A-PK       xG npxG   xA             xG    xA xG+xA  npxG npxG+xA   
0  0.07   0.14      2.4  1.6  3.2           0.08  0.11  0.20  0.06    0.17   
1  0.00   0.03      0.5  0.5  3.4           0.01  0.09  0.11  0.01    0.11   
2  0.11   0.11      1.3  1.3  0.1           0.05  0.00  0.05  0.05    0.05   
3  0.57   0.57      1.3  1.3  0.2           0.72  0.10  0.82  0.72    0.82   
4  0.05   0.15      1.6  1.6  1.4           0.08  0.07  0.15  0.08    0.15   

  Unnamed: 30_level_0  
              Matches  
0             Matches  
1             Matches  
2             Matches  
3             Matches  
4             Matches

#### Column selection 
**driven by domain knowledge + aggregation enabling (total columns reduced from 118 to 45)**
  

*overall:* **Unnamed:** Player, Nation, Pos, Age; **Playing Time:** Min; Performance, **Performance:** Gls, Ast; **Expected:** take All; **Per 90 Minutes:** ignore All         
*chance_creation:* **SCA:** SCA; **SCA Types:** take ALL; **GCA:** GCA; **GCA Types:** take All   
*defense:* **Vs Dribbles:** ignore ALL; **Tackles:** Tkl, TklW;  **Pressures**: Press, Succ; **Blocks:** Blocks, Int, Clr, Err  
*passing:* **Total:** Cmp, Att, PrgDist; **Long:** Prog     
*possession:* **Touches:** Live; **Dribbles:** Att, #Pl; **Carries:** TotDist, PrgDist; **Receiving:** Targ,Rec






In [725]:
overall_subset = overall[[('Unnamed: 0_level_0', 'Rk'), ('Unnamed: 1_level_0', 'Player'), ('Unnamed: 2_level_0', 'Nation'),
                          ('Unnamed: 3_level_0', 'Pos'),('Unnamed: 4_level_0','Squad'),('Unnamed: 5_level_0', 'Comp'),
                          ('Unnamed: 6_level_0', 'Age'),('Playing Time', 'Min'),('Performance', 'Gls'),('Performance', 'Ast'),
                          ('Expected', 'xG'),('Expected', 'npxG'), ('Expected', 'xA')]]
chance_creation_subset = chance_creation[[('SCA', 'SCA'), ('SCA Types', 'PassLive'), ('SCA Types', 'PassDead'),
                                        ('SCA Types', 'Drib'), ('SCA Types','Sh'), ('SCA Types', 'Fld'), ('SCA Types', 'Def'),
                                        ('GCA', 'GCA'), ('GCA Types', 'PassLive'), ('GCA Types', 'PassDead'), 
                                        ('GCA Types','Drib'), ('GCA Types','Sh'), ('GCA Types','Fld'), ('GCA Types','Def'),
                                        ('GCA Types','OG')]]
defense_subset = defense[[('Tackles','Tkl'), ('Tackles','TklW'),('Pressures', 'Press'), ('Pressures', 'Succ'), 
                          ('Blocks','Blocks'), ('Unnamed: 28_level_0', 'Int'),('Unnamed: 29_level_0', 'Tkl+Int'),
                          ('Unnamed: 30_level_0', 'Clr')]]
passing_subset = passing[[('Total', 'Cmp'), ('Total','Att'),('Total', 'PrgDist'),('Unnamed: 30_level_0','Prog')]]
possession_subset = possession[[('Touches', 'Live'), ('Dribbles', 'Att'), ('Dribbles', '#Pl'),( 'Carries', 'TotDist'),
                                ( 'Carries', 'PrgDist'),( 'Receiving', 'Targ'),( 'Receiving', 'Rec')]]


# Merge
we combine the reduce and lambda functions to merge all 5 subset dfs in one go

In [726]:
data_frames = [overall_subset, chance_creation_subset, defense_subset, passing_subset, possession_subset]
merged = reduce(lambda left, right: pd.merge(left, right[right.columns.difference(left.columns)],
                                         left_index=True, right_index=True), data_frames)

In [727]:
#check the merge works as intended
# merged.head()

# Clean

#### Format

In [728]:
#cleaning format of Player
merged[('Unnamed: 1_level_0' ,'Player')] = merged[('Unnamed: 1_level_0' ,'Player')].str.split('\\', expand=True)[0]
#cleaning format of Nation
merged[('Unnamed: 2_level_0', 'Nation')] = merged[('Unnamed: 2_level_0', 'Nation')].str.split(' ', expand=True)[1]
#cleaning format of Position
merged[('Unnamed: 3_level_0', 'Pos')] = merged[('Unnamed: 3_level_0', 'Pos')].str[0:2]

#### Data type 

In [729]:
#changing Age from float to int64
merged[('Unnamed: 6_level_0', 'Age')].fillna(0, inplace=True) #else you cannot conver to int due to NA values
merged[('Unnamed: 6_level_0', 'Age')] = merged[('Unnamed: 6_level_0', 'Age')].astype(np.int64)

#changing Squad from Object to String type to facilitate groupby sum of duplicates

merged[('Unnamed: 5_level_0', 'Comp')] = merged[('Unnamed: 5_level_0', 'Comp')].astype(str)


#### Column names

In [730]:
#Removing level 0 column names and renaming level 1 columnnames to avoid confusion

#to avoid naming conflict during rename procedure (many level 1 columns have same name and its not possible to input
# multiindex column names into the rename function), we have to make sure each columnname name is unique before renaming 
# by combining level 0 and level 1 column names.
merged.columns = merged.columns.to_flat_index() 

# we follow the steps below to automize the renaming procedure
dic = {"('GCA Types', 'Def')":'Def_GCA', "('GCA Types', 'Drib')":'Drib_GCA', "('GCA Types', 'Fld')":'Fld_GCA', 
       "('GCA Types', 'OG')":'OG_GCA', "('GCA Types', 'PassDead')":"PassDead", "('GCA Types', 'PassLive')":'PassLive_GCA',
       "('GCA Types', 'Sh')":'Sh_SCA', "('SCA Types', 'Def')":'Def_SCA', "('SCA Types', 'Drib')":'Drib_SCA',
       "('SCA Types', 'Fld')":'Fld_SCA', "('SCA Types', 'PassDead')":'PassDead_SCA', "('SCA Types', 'PassLive')":'PassLive_SCA',
       "('SCA Types', 'Sh')":'Sh_SCA', "('Pressures', 'Succ')":'Succ_Press', "('Total', 'Att')":'Att_Pass', 
       "('Total', 'Cmp')": 'Cmp_Pass', "('Total', 'PrgDist')":'TotPrgDist_Pass', "('Unnamed: 30_level_0', 'Prog')":'#Prog_Pass',
       "('Carries', 'PrgDist')":'TotPrgDist_Carried', "('Carries', 'TotDist')":'TotDist_Carried', "('Dribbles', '#Pl')":'Oppon_Drib',
       "('Dribbles', 'Att')":'Att_Drib', "('Receiving', 'Rec')":'Recep', "('Receiving', 'Targ')":'Att_Recep', 
       "('Touches', 'Live')": 'Live_Touches'}

cols_flat = list(merged.columns)
cols_new = []
for i in cols_flat:
    if str(i) in dic:
        cols_new.append(dic[str(i)])
    else:
        cols_new.append(i[1])
        
renaming_dict = dict(zip(cols_flat, cols_new))
merged.rename(columns=renaming_dict, inplace=True)

In [731]:
merged.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Min,Gls,Ast,xG,npxG,xA,GCA,Def_GCA,Drib_GCA,Fld_GCA,OG_GCA,PassDead,PassLive_GCA,Sh_SCA,SCA,Def_SCA,Drib_SCA,Fld_SCA,PassDead_SCA,PassLive_SCA,Sh_SCA,Blocks,Press,Succ_Press,Tkl,TklW,Int,Tkl+Int,Clr,Att_Pass,Cmp_Pass,TotPrgDist_Pass,#Prog_Pass,TotPrgDist_Carried,TotDist_Carried,Oppon_Drib,Att_Drib,Recep,Att_Recep,Live_Touches
0,1,Patrick van Aanholt,NED,DF,Crystal Palace,eng Premier League,28,2507,3,2,2.4,1.6,3.2,5.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,62.0,0.0,3.0,2.0,23.0,32.0,2.0,57.0,267.0,74.0,51.0,32,23,74,67.0,1558.0,1227.0,9280.0,129.0,3129.0,5651.0,18.0,36.0,994.0,1079.0,1458.0
1,2,Max Aarons,ENG,DF,Norwich City,eng Premier League,19,3240,0,1,0.5,0.5,3.4,3.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,55.0,1.0,3.0,4.0,2.0,42.0,3.0,100.0,487.0,132.0,64.0,36,20,84,86.0,1684.0,1316.0,6834.0,77.0,5439.0,9489.0,55.0,94.0,1244.0,1399.0,1821.0
2,3,Yunis Abdelhamid,MAR,DF,Reims,fr Ligue 1,31,2520,3,0,1.3,1.3,0.1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,16.0,0.0,0.0,1.0,0.0,11.0,4.0,43.0,299.0,111.0,61.0,37,24,85,166.0,1515.0,1328.0,9041.0,80.0,4306.0,7673.0,21.0,26.0,1236.0,1266.0,1782.0
3,4,Suleiman Abdullahi,NGA,FW,Union Berlin,de Bundesliga,22,157,1,0,1.3,1.3,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,25.0,9.0,1.0,1,1,2,2.0,40.0,18.0,132.0,3.0,177.0,279.0,3.0,4.0,39.0,77.0,62.0
4,5,Mehdi Abeid,ALG,MF,Nantes,fr Ligue 1,26,1817,1,2,1.6,1.6,1.4,4.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,46.0,1.0,1.0,1.0,14.0,27.0,2.0,18.0,417.0,114.0,60.0,41,17,77,22.0,957.0,801.0,3850.0,59.0,2116.0,4298.0,12.0,22.0,759.0,854.0,1091.0


#### Duplicates

In [732]:
#the code below confirms that all the duplicated rows we are dealing with are due to a player changing squads mid season
print(Counter(merged.duplicated(subset=['Player', 'Age'])))
print(Counter(merged.duplicated(subset=['Player', 'Age','Nation', 'Squad'])))
# merged[merged.duplicated(subset=['Player', 'Age'], keep=False)].sort_values('Player')

Counter({False: 2613, True: 119})
Counter({False: 2732})


In [733]:
# because of our target analysis (realtionships between a players performance stats and his market value) we want to have 
# each individual player stats aggregated always into a single row instead of spread across two because of the two clubs
# he played for. 

#look at a and b output dfs to understand better how the code below works
a = merged.groupby(by=['Player', 'Nation', 'Age']).agg({'Squad':' / '.join, 'Comp':' / '.join}, )
b = merged.groupby(by=['Player', 'Nation', 'Age']).sum()
merged_clean = pd.merge(a, b, on= ['Player', 'Nation', 'Age'], how='inner')

#bring table back to original order
merged_clean = merged_clean.reset_index().sort_values('Rk')

In [734]:
#players that changed club mid season but stayed in the same league have now under comp the same league twice
#separated by a '/'. This is redundant info, thus the code below removes the duplicated league after the / for those cases
merged_clean['Comp']=np.where((merged_clean['Comp'].str.contains("/")) &
          (merged_clean['Comp'].str.split('/', expand=True)[0].str.strip() ==
           merged_clean['Comp'].str.split('/', expand=True)[1].str.strip()),
         merged_clean['Comp'].str.split('/', expand=True)[0], merged_clean['Comp']
        )

In [737]:
#we flag the players which played in two different leagues within the same season. We will not use
# them in our analysis because they are a negligible amount of observations (64 out ofm roughly 3000 observations) 
#and make the task of answering our question regarding differences among the value attached to certain peformance metrics 
#across leagues significantly more complex to answer.
merged_clean['played_two_leagues'] = np.where(merged_clean['Comp'].str.contains("/"), True, False)
print("# players played in two different leagues: " + str(sum(merged_clean['played_two_leagues'])))
print(merged_clean.shape)

# players played in two different leagues: 48
(2612, 47)


In [738]:
merged_clean.head()

,Player,Nation,Age,Squad,Comp,Rk,Min,Gls,Ast,xG,npxG,xA,GCA,Def_GCA,Drib_GCA,Fld_GCA,OG_GCA,PassDead,PassLive_GCA,Sh_SCA,SCA,Def_SCA,Drib_SCA,Fld_SCA,PassDead_SCA,PassLive_SCA,Sh_SCA,Blocks,Press,Succ_Press,Tkl,TklW,Int,Tkl+Int,Clr,Att_Pass,Cmp_Pass,TotPrgDist_Pass,#Prog_Pass,TotPrgDist_Carried,TotDist_Carried,Oppon_Drib,Att_Drib,Recep,Att_Recep,Live_Touches,played_two_leagues
1943,Patrick van Aanholt,NED,28,Crystal Palace,eng Premier League,1,2507,3,2,2.4,1.6,3.2,5.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,62.0,0.0,3.0,2.0,23.0,32.0,2.0,57.0,267.0,74.0,51.0,32,23,74,67.0,1558.0,1227.0,9280.0,129.0,3129.0,5651.0,18.0,36.0,994.0,1079.0,1458.0,False
1696,Max Aarons,ENG,19,Norwich City,eng Premier League,2,3240,0,1,0.5,0.5,3.4,3.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,55.0,1.0,3.0,4.0,2.0,42.0,3.0,100.0,487.0,132.0,64.0,36,20,84,86.0,1684.0,1316.0,6834.0,77.0,5439.0,9489.0,55.0,94.0,1244.0,1399.0,1821.0,False
2553,Yunis Abdelhamid,MAR,31,Reims,fr Ligue 1,3,2520,3,0,1.3,1.3,0.1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,16.0,0.0,0.0,1.0,0.0,11.0,4.0,43.0,299.0,111.0,61.0,37,24,85,166.0,1515.0,1328.0,9041.0,80.0,4306.0,7673.0,21.0,26.0,1236.0,1266.0,1782.0,False
2330,Suleiman Abdullahi,NGA,22,Union Berlin,de Bundesliga,4,157,1,0,1.3,1.3,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,25.0,9.0,1.0,1,1,2,2.0,40.0,18.0,132.0,3.0,177.0,279.0,3.0,4.0,39.0,77.0,62.0,False
1715,Mehdi Abeid,ALG,26,Nantes,fr Ligue 1,5,1817,1,2,1.6,1.6,1.4,4.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,46.0,1.0,1.0,1.0,14.0,27.0,2.0,18.0,417.0,114.0,60.0,41,17,77,22.0,957.0,801.0,3850.0,59.0,2116.0,4298.0,12.0,22.0,759.0,854.0,1091.0,False


# Save

In [736]:
merged_clean.to_csv('fbref_player_statistics.csv')